In [1]:
import pandas as pd

m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv("/Users/sihoon/Desktop/ml-10M100K/movies.dat", names=m_cols, sep='::', encoding='latin-1', engine='python') # m_cols : m_cols라는 컬럼의 값들을 컬럼명으로 사용, engine : 원래 pandas가 c를 기준으로 좋음 파이썬으로 정의

movies['genre'] = movies.genre.apply(lambda  x : x.split('|'))
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [2]:
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('/Users/sihoon/Desktop/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()

user_tagged_movies.head()

,user_id,movie_id,tag,timestamp
0,15,4973,excellent!,1215184630
1,20,1747,politics,1188263867
2,20,1747,satire,1188263867
3,20,2424,chick flick 212,1188263835
4,20,2424,hanks,1188263835


In [3]:
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list}) # tag를 영화별로 list 형식으로 저장
movies = movies.merge(movie_tags, on='movie_id', how='left')

movies.head()

,movie_id,title,genre,tag
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat..."
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro..."
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra..."
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie]
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin..."


**이 아래부분의 경우, 데이터의 양이 많아 시간이 오래 걸린다.**

In [4]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/sihoon/Desktop/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


**이 아래처럼 실무에서는 이렇게 작게 나눠서 실험을 반복한다.**

In [5]:
valid_user_ids= sorted(ratings.user_id.unique())[:1000] # 우선 1000개의 데이터만
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]

movielens = ratings.merge(movies, on='movie_id')
movielens.head()

,user_id,movie_id,rating,timestamp,title,genre,tag
0,1,122,5.0,838985046,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
1,139,122,3.0,974302621,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
2,149,122,2.5,1112342322,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
3,182,122,3.0,943458784,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."
4,215,122,4.5,1102493547,Boomerang (1992),"[Comedy, Romance]","[dating, nudity (topless - brief), can't remem..."


---

In [6]:
import numpy as np
movielens.groupby('user_id').agg({'movie_id' : len}).agg({'movie_id' : [min, max, np.mean, len]})

'''
가장 평가 수가 적은 사용자는 20 작품에 평가
가장 평가 수가 많은 사용자는 1668 작품에 평가
각 사용자는 평균 139 작품에 평가
평가한 모든 사용자는 1000명
'''

'\n가장 평가 수가 적은 사용자는 20 작품에 평가\n가장 평가 수가 많은 사용자는 1668 작품에 평가\n각 사용자는 평균 139 작품에 평가\n평가한 모든 사용자는 1000명\n'

In [7]:
import numpy as np
movielens.groupby('movie_id').agg({'movie_id' : len}).agg({'movie_id' : [min, max, np.mean, len]})

'''
가장 평가 수가 적은 영화는 1명이 평가
가장 평가 수가 많은 영화는 496명이 평가
각 영화는 평균 20명이 평가
평가된 모든 영화는 6736 작품
'''

'\n가장 평가 수가 적은 영화는 1명이 평가\n가장 평가 수가 많은 영화는 496명이 평가\n각 영화는 평균 20명이 평가\n평가된 모든 영화는 6736 작품\n'

In [8]:
movielens['timestamp_rank'] = movielens.groupby('user_id')['timestamp'].rank(ascending=False, method='first')
'''
학습용과 테스트용으로 데이터를 분할
각 사용자가 가장 최근에 평가한 5건의 영화를 평가용으로 사용하고 그 외에는 학습용으로 사용
먼적 각 사용자가 평가한 영화의 순서를 계산
직전에 평가한 영화부터 순서를 부여해 나간다. (1에서 시작)

method='first'를 사용해 동일한 값 중에서 먼저 나타나는 것에 더 낮은 순위를 부여합니다.
method='first'는 rank 메서드에서 동일한 값이 있는 경우에 순위를 부여하는 방식을 지정하는 매개변수입니다.

'''
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank'] <= 5]

***학습용 데이터를 사용해 사용자가 가장 최근에 평가한 5개 영화의 평갓값을 얼마나 정확하게 예측할 수 있는가로 추천 알고리즘의 성능을 평가한다. 그 지표는 RMSE를 사용
예측이 실제 평갓값과 완전히 일치하면 RMSE는 0이 된다. RMSE가 작을수록 추천 알고리즘의 성능이 좋다고 볼 수 있다.***

In [9]:
from typing import List
from sklearn.metrics import mean_squared_error
def calc_rmse(self, true_rating: List[float], pred_rating: List[float]) -> float:
    return np.sqrt(mean_squared_error(true_rating, pred_rating))

'''
위 코드는 파이썬에서 구현된 평균 제곱 오차 (Mean Squared Error, MSE)를 계산하고 이를 제곱근을 취해 반환하는 함수입니다. 이 함수는 회귀 문제에서 모델의 예측값과 실제값 간의 평균 제곱 오차를 평가하는 데 사용됩니다.

함수의 입력은 두 개의 리스트인 true_rating과 pred_rating입니다. true_rating은 모델을 평가할 실제 값들을 포함하는 리스트이고, pred_rating은 모델이 예측한 값들을 포함하는 리스트입니다.

함수는 mean_squared_error 함수를 사용하여 두 리스트 간의 평균 제곱 오차를 계산합니다. 그 후, 이 값을 np.sqrt 함수를 사용하여 제곱근을 취하여 최종적으로 평균 제곱근 오차(RMSE)를 계산하고 반환합니다.

'''

'\n위 코드는 파이썬에서 구현된 평균 제곱 오차 (Mean Squared Error, MSE)를 계산하고 이를 제곱근을 취해 반환하는 함수입니다. 이 함수는 회귀 문제에서 모델의 예측값과 실제값 간의 평균 제곱 오차를 평가하는 데 사용됩니다.\n\n함수의 입력은 두 개의 리스트인 true_rating과 pred_rating입니다. true_rating은 모델을 평가할 실제 값들을 포함하는 리스트이고, pred_rating은 모델이 예측한 값들을 포함하는 리스트입니다.\n\n함수는 mean_squared_error 함수를 사용하여 두 리스트 간의 평균 제곱 오차를 계산합니다. 그 후, 이 값을 np.sqrt 함수를 사용하여 제곱근을 취하여 최종적으로 평균 제곱근 오차(RMSE)를 계산하고 반환합니다.\n\n'